In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
tf.__version__

'2.0.0'

# preparing training dataframe


In [3]:
training_set = pd.read_csv('train.csv')

In [4]:
training_set

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0
...,...,...
59995,59996,5
59996,59997,1
59997,59998,3
59998,59999,0


In [5]:
training_img = ["{}.png".format(x) for x in list(training_set.id)]

In [6]:
training_label = list(training_set.label)

In [7]:
dict = {'Images': training_img,
       'labels': training_label}

In [8]:
training_set = pd.DataFrame(dict)

In [9]:
training_set

,Images,labels
0,1.png,9
1,2.png,0
2,3.png,0
3,4.png,3
4,5.png,0
...,...,...
59995,59996.png,5
59996,59997.png,1
59997,59998.png,3
59998,59999.png,0


In [10]:
training_set.Images = training_set.Images.astype(str)
training_set.labels = training_set.labels.astype(str)

In [11]:
training_set.dtypes

Images    object
labels    object
dtype: object

# preparing test dataframe

In [51]:
test_set = pd.read_csv('test.csv')

In [52]:
test_img = ["{}.png".format(x) for x in list(test_set.id)]

In [53]:
dict2 = {'id':test_img}

In [54]:
test_set = pd.DataFrame(dict2)

In [55]:
test_set

,id
0,60001.png
1,60002.png
2,60003.png
3,60004.png
4,60005.png
...,...
9995,69996.png
9996,69997.png
9997,69998.png
9998,69999.png


In [56]:
len(test_set)

10000

# preprocessing the tarining image

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)



In [22]:
train_generator = train_datagen.flow_from_dataframe(training_set,
                                                   directory='train',
                                                   x_col='Images',
                                                   y_col='labels',
                                                   class_mode='categorical',
                                                   target_size=(64,64),
                                                   batch_size=32)


                                                  
                                                 

Found 60000 validated image filenames belonging to 10 classes.


# building convolution neural network

In [23]:
classifier = Sequential()

In [24]:
#first convolutional layer
classifier.add(Conv2D(filters=56, kernel_size=(3,3), activation='relu',
                     input_shape=(64,64,3)))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#second convolution layer
classifier.add(Conv2D(32,(3,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [25]:
#flattening
classifier.add(Flatten())

In [26]:
#hidden layer
classifier.add(Dense(units=64,activation='relu'))

#output layer
classifier.add(Dense(units=10, activation='softmax'))

In [27]:
classifier.compile(optimizer = 'adam', loss='categorical_crossentropy', 
                   metrics=['categorical_accuracy','accuracy'])

In [28]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 56)        1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 56)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        16160     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                401472    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6

# training

In [29]:
classifier.fit_generator(train_generator, epochs=50, steps_per_epoch=60)

Epoch 1/50
60/60 [==============================] - 12s 196ms/step - loss: 1.2588 - categorical_accuracy: 0.5464 - accuracy: 0.5464
Epoch 2/50
60/60 [==============================] - 11s 191ms/step - loss: 0.7303 - categorical_accuracy: 0.7375 - accuracy: 0.73756s - loss: 0.7122 - categorical_ac
Epoch 3/50
60/60 [==============================] - 11s 187ms/step - loss: 0.6775 - categorical_accuracy: 0.7479 - accuracy: 0.74790s - loss: 0.6848 - categorical_accuracy: 0.7446 - accuracy: 0.74
Epoch 4/50
60/60 [==============================] - 11s 190ms/step - loss: 0.6301 - categorical_accuracy: 0.7776 - accuracy: 0.7776
Epoch 5/50
60/60 [==============================] - 12s 199ms/step - loss: 0.5892 - categorical_accuracy: 0.7901 - accuracy: 0.7901
Epoch 6/50
60/60 [==============================] - 12s 193ms/step - loss: 0.6148 - categorical_accuracy: 0.7786 - accuracy: 0.7786
Epoch 7/50
60/60 [==============================] - 12s 198ms/step - loss: 0.5364 - categorical_accuracy: 0.8

In [32]:
from tensorflow.keras.preprocessing import image

In [33]:
Y_predic = []
for i in range(len(test_set)):
    img = image.load_img(path='test/{}'.format(test_set.Images[i]), target_size=(64,64,3))
    img = image.img_to_array(img)
    img_test = img.reshape((1,64,64,3))
    classifier.predict_classes(img_test)
    Y_predic.append(classifier.predict_classes(img_test))

In [57]:
test_set['labels']=Y_predic

In [58]:
test_set

,id,labels
0,60001.png,[9]
1,60002.png,[2]
2,60003.png,[1]
3,60004.png,[1]
4,60005.png,[6]
...,...,...
9995,69996.png,[9]
9996,69997.png,[1]
9997,69998.png,[8]
9998,69999.png,[1]


In [59]:
test_set.to_csv('sample_submission.csv')